# Simple Video RAG Search Pipeline
## Direct Upload - No CSV Required

This notebook provides a streamlined pipeline:
1. **Extract video metadata** from files
2. **Create RAG corpus** using Vertex AI
3. **Upload directly** using rag.upload_file()
4. **Search and query** your video content

### Prerequisites
- Google Cloud Project with Vertex AI enabled
- Video files in a local directory

## Setup and Configuration

In [1]:
# Install required packages
# !pip install google-cloud-aiplatform google-cloud-storage google-genai pandas pathlib --upgrade --quiet

print("✅ All packages installed successfully!")

✅ All packages installed successfully!


In [2]:
# Import libraries

from google import genai
from google.genai import types
import json
import os
import uuid
import pandas as pd
import vertexai
from vertexai import rag
from vertexai.generative_models import GenerativeModel, Tool
from google.cloud import storage
from pathlib import Path
import re
import time
import datetime
import shutil

# Import Google Genai for video analysis (optional)
try:
    from google import genai
    from google.genai.types import Content, GenerateContentConfig, Part
    print("🎬 Gemini video analysis available")
except ImportError:
    print("⚠️ Gemini components not available - proceeding with metadata only")

print("✅ All libraries imported successfully!")

🎬 Gemini video analysis available
✅ All libraries imported successfully!


In [3]:
# Configuration - UPDATE THESE VALUES
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0] if PROJECT_ID else "your-project-id"  # UPDATE THIS

REGION = "us-central1"
VIDEO_FOLDER = "video"  # UPDATE: Your video folder path
EMBEDDING_MODEL = 'text-multilingual-embedding-002'

# RAG Configuration
today = datetime.date.today()
RAG_CORPUS_NAME = "video_content_corpus_" + today.strftime("%Y-%m-%d")
BUCKET_NAME = f"{PROJECT_ID}-video-rag-data"

# Initialize Vertex AI
vertexai.init(project=PROJECT_ID, location=REGION)

# Configuration
BUCKET_NAME = "my-project-0004-346516-video-rag-data"
VIDEO_FOLDER_PREFIX = "video_analysis/"

print(f"✅ Configuration complete!")
print(f"📁 Project ID: {PROJECT_ID}")
print(f"📁 Video Folder: {VIDEO_FOLDER}")
print(f"📊 RAG Corpus: {RAG_CORPUS_NAME}")

✅ Configuration complete!
📁 Project ID: my-project-0004-346516
📁 Video Folder: video
📊 RAG Corpus: video_content_corpus_2025-09-25


## Extract Video Metadata

In [4]:
from google import genai
from google.genai import types
import json
import uuid
import pandas as pd
import re
from pathlib import Path
from google.cloud import storage
import time
import os

# Configuration
PROJECT_ID = "my-project-0004-346516"
BUCKET_NAME = "my-project-0004-346516-video-rag-data"
VIDEO_FOLDER_PREFIX = "video_analysis/"
TEXT_FILES_PREFIX = "processed_text_files_scene/"  # Where to store analysis text files
JSONL_OUTPUT_FILE = "video_documents_scene.jsonl"

# Your retry-enabled Gemini function (included as provided)
def generate_video_analysis(system_instructions, main_prompt, file_uri):
    """Generate comprehensive video analysis using Gemini 2.5 Pro with exponential retry."""
    
    max_retries = 6
    base_delay = 2  # Start with 2 seconds
    max_delay = 300  # Maximum 5 minutes
    
    for attempt in range(max_retries + 1):
        try:
            client = genai.Client(
                vertexai=True,
                project=PROJECT_ID,
                location="global",
            )
            
            msg1_video1 = types.Part.from_uri(
                file_uri=file_uri,
                mime_type="video/mp4",
            )
            
            text1 = types.Part.from_text(text=main_prompt)
            system_prompt = types.Part.from_text(text=system_instructions)
            
            model = "gemini-2.5-pro"
            contents = [
                types.Content(
                    role="user",
                    parts=[msg1_video1, text1]
                ),
            ]
            
            generate_content_config = types.GenerateContentConfig(
                temperature=1,
                top_p=1,
                seed=0,
                max_output_tokens=65535,
                safety_settings=[
                    types.SafetySetting(category="HARM_CATEGORY_HATE_SPEECH", threshold="OFF"),
                    types.SafetySetting(category="HARM_CATEGORY_DANGEROUS_CONTENT", threshold="OFF"),
                    types.SafetySetting(category="HARM_CATEGORY_SEXUALLY_EXPLICIT", threshold="OFF"),
                    types.SafetySetting(category="HARM_CATEGORY_HARASSMENT", threshold="OFF")
                ],
                response_mime_type="application/json",
                response_schema={"type": "OBJECT", "properties": {"response": {"type": "STRING"}}},
                system_instruction=[system_prompt],
            )
            
            # Initialize response collection
            full_response_text = []
            
            if attempt == 0:
                print(f"🤖 Starting Gemini analysis for: {file_uri}")
            else:
                print(f"🔄 Retry attempt {attempt} for: {file_uri}")
            
            response_stream = client.models.generate_content_stream(
                model=model,
                contents=contents,
                config=generate_content_config,
            )
            
            for chunk in response_stream:
                if chunk.text:
                    full_response_text.append(chunk.text)
                    print(".", end="", flush=True)  # Progress indicator
            
            print(" ✅ Analysis complete!")
            
            # Join the chunks to get the full response string
            final_json_string = "".join(full_response_text)
            
            if not final_json_string:
                print("Warning: Received an empty response from the model.")
                return None
                
            return final_json_string
            
        except Exception as e:
            error_str = str(e)
            
            # Check if this is a 429 RESOURCE_EXHAUSTED error
            if "429" in error_str and "RESOURCE_EXHAUSTED" in error_str:
                if attempt < max_retries:
                    # Calculate exponential backoff delay
                    delay = min(base_delay * (2 ** attempt), max_delay)
                    
                    print(f"")  # New line after progress dots
                    print(f"⚠️  Rate limit hit (attempt {attempt + 1}/{max_retries + 1})")
                    print(f"⏱️  Waiting {delay} seconds before retry...")
                    
                    time.sleep(delay)
                    continue
                else:
                    print(f"")  # New line after progress dots
                    print(f"❌ Max retries ({max_retries}) exceeded for rate limiting")
                    return None
            else:
                # For non-rate-limit errors, don't retry
                print(f"")  # New line after progress dots
                print(f"❌ Non-retryable error in Gemini analysis: {e}")
                return None
    
    # Should never reach here
    return None

def analyze_filename_content(filename):
    """Enhanced filename analysis for metadata extraction."""
    analysis = {
        'genre': 'General',
        'content_type': 'Video Content',
        'language': 'Unknown',
        'category': 'General',
        'keywords_en': [],
        'keywords_id': [],
        'detailed_analysis': ''
    }
    
    # Romance/Drama Series Analysis
    if "Cinta Sedalam Rindu" in filename:
        analysis.update({
            'genre': 'Romance/Drama',
            'content_type': 'Indonesian Drama Series',
            'language': 'Bahasa Indonesia',
            'category': 'Entertainment',
            'keywords_en': ['romance', 'love story', 'drama', 'series', 'episode'],
            'keywords_id': ['cinta', 'rindu', 'drama', 'serial', 'episode']
        })
        
        episode_match = re.search(r'Episode\s*(\d+)', filename)
        if episode_match:
            analysis['detailed_analysis'] += f"Episode Number: {episode_match.group(1)}\n"
        
        if "Galaxy" in filename:
            analysis['detailed_analysis'] += "Main Character: Galaxy\n"
            analysis['keywords_en'].append('Galaxy')
        if "Aluna" in filename:
            analysis['detailed_analysis'] += "Main Character: Aluna\n"
            analysis['keywords_en'].append('Aluna')
    
    # Sports Content Analysis
    elif "Highlight" in filename:
        analysis.update({
            'genre': 'Sports',
            'content_type': 'Football/Soccer Highlights',
            'language': 'Bahasa Indonesia',
            'category': 'Sports Entertainment',
            'keywords_en': ['football', 'soccer', 'sports', 'match', 'highlight'],
            'keywords_id': ['sepak bola', 'pertandingan', 'highlight', 'olahraga']
        })
        
        pekan_match = re.search(r'Pekan\s*(\d+)', filename)
        if pekan_match:
            analysis['detailed_analysis'] += f"Match Week: {pekan_match.group(1)}\n"
        
        vs_match = re.search(r'([A-Za-z\s]+)\s+VS\s+([A-Za-z\s]+)', filename)
        if vs_match:
            team1, team2 = vs_match.groups()
            analysis['detailed_analysis'] += f"Teams: {team1.strip()} vs {team2.strip()}\n"
    
    # News Content Analysis
    elif "Liputan 6" in filename:
        analysis.update({
            'genre': 'News/Journalism',
            'content_type': 'News Report',
            'language': 'Bahasa Indonesia',
            'category': 'News & Current Affairs',
            'keywords_en': ['news', 'journalism', 'report', 'current affairs'],
            'keywords_id': ['berita', 'liputan', 'jurnalisme', 'laporan']
        })
        analysis['detailed_analysis'] += "News Channel: Liputan 6\n"
    
    analysis['keywords_en'].extend(['video', 'content', 'media'])
    analysis['keywords_id'].extend(['video', 'konten', 'media'])
    
    return analysis

def get_gcs_video_files(bucket_name, folder_prefix):
    """Get list of video files from GCS bucket."""
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    
    video_extensions = ['.mp4', '.avi', '.mov', '.mkv', '.webm']
    video_files = []
    
    for blob in bucket.list_blobs(prefix=folder_prefix):
        if any(blob.name.lower().endswith(ext) for ext in video_extensions):
            video_files.append(blob)
    
    print(f"🎬 Found {len(video_files)} video files in GCS bucket")
    return video_files

def upload_text_to_gcs(content, filename, bucket_name, prefix):
    """Upload text content to GCS and return the URI."""
    try:
        client = storage.Client()
        bucket = client.bucket(bucket_name)
        
        # Create blob path
        blob_path = f"{prefix}{filename}"
        blob = bucket.blob(blob_path)
        
        # Upload content
        blob.upload_from_string(content, content_type='text/plain')
        
        # Return GCS URI
        gcs_uri = f"gs://{bucket_name}/{blob_path}"
        return gcs_uri
        
    except Exception as e:
        print(f"❌ Error uploading {filename}: {e}")
        return None

def extract_structured_metadata(filename, content_analysis):
    """Extract structured metadata for JSONL."""
    
    # Extract episode information
    episode = None
    episode_match = re.search(r'Episode\s*(\d+)', filename)
    if episode_match:
        episode = episode_match.group(1)
    
    # Extract series/show name
    series_name = None
    if "Cinta Sedalam Rindu" in filename:
        series_name = "Cinta Sedalam Rindu"
    elif "Liputan 6" in filename:
        series_name = "Liputan 6"
    
    # Extract characters
    characters = []
    if "Galaxy" in filename:
        characters.append("Galaxy")
    if "Aluna" in filename:
        characters.append("Aluna")
    
    # Extract teams for sports content
    teams = []
    vs_match = re.search(r'([A-Za-z\s]+)\s+VS\s+([A-Za-z\s]+)', filename)
    if vs_match:
        teams = [vs_match.group(1).strip(), vs_match.group(2).strip()]
    
    # Generate year (default to 2024, can be enhanced)
    year = "2024"
    
    return {
        'episode': episode,
        'series_name': series_name,
        'characters': characters,
        'teams': teams,
        'year': year
    }

def complete_video_processing_pipeline():
    """Complete pipeline: GCS videos -> Gemini analysis -> Text files -> JSONL."""
    
    print("🚀 STARTING COMPLETE VIDEO PROCESSING PIPELINE")
    print("=" * 80)
    
    # System instructions for Gemini
    system_instructions = """You are an expert video content analyzer specializing in Indonesian entertainment, news, and sports content.

Analyze this video comprehensively and provide detailed information including:

**DIALOGUE TRANSCRIPTION:**
- NOT needed

**DETAILED SCENE DESCRIPTIONS:**
- Describe specific actions, movements, and events with precise timestamps
- Include character interactions and relationships
- Note visual elements: clothing, settings, objects, expressions, backgrounds
- Describe sports actions: goals, fouls, cards, celebrations with exact timing
- Include news events: interviews, locations, demonstrations, investigations with context
- Mention specific names, places, and identifying details

**NEWS CONTENT - CRITICAL DETAILS EXTRACTION:**
- ADDRESSES AND LOCATIONS: Extract house numbers, street names, building names, area names, city districts
- VISUAL TEXT ELEMENTS: Read and transcribe any visible text, banners, signs, posters, building numbers, street signs
- PERSON IDENTIFICATION: Name everyone shown or mentioned - officials, witnesses, suspects, victims, demonstrators
- SPECIFIC STATEMENTS: Record exact quotes, especially from officials, politicians, or key figures
- PHYSICAL EVIDENCE: Describe objects, vehicles, buildings, damage, weapons, or evidence shown
- OFFICIAL INFORMATION: Note police statements, government responses, investigation details
- INCIDENT DETAILS: Precise descriptions of what happened, where, when, and involving whom
- BUILDING AND STRUCTURE DETAILS: Architecture, floors, rooms, entry/exit points, security features

**CHARACTER AND PERSON IDENTIFICATION:**
- Name all people mentioned or shown (actors, athletes, officials, reporters, witnesses)
- Describe their roles, relationships, and actions
- Note character development and story progression  
- Include sports player names, team affiliations, and positions
- For news: Include official titles, positions, and affiliations

**SPECIFIC EVENT DETAILS:**
- Document exact sequences of events with precise timing
- Include score information, match events, and outcomes for sports
- Note news story developments and key facts with timestamps
- Describe dramatic moments, conflicts, and resolutions with timing
- For investigations: Timeline of events, evidence collection, official responses

**VISUAL ANALYSIS:**
- Describe everything visible in each scene: backgrounds, objects, people, text, symbols

**SEARCHABLE CONTENT CREATION:**
Create extremely detailed, searchable descriptions that capture every moment someone might want to find:
- "At [timestamp], character X does specific action Y"
- "At [timestamp], sports event X happens involving player Y"
- "At [timestamp], news event X occurs at specific location Y"
- "At [timestamp], visible text shows Z information"
- "At [timestamp], building number/address X is visible/mentioned"

Focus on creating rich, timestamped, searchable content that captures EVERY detail, no matter how minor it seems."""


    main_prompt = """Analyze this video completely and provide scene descriptions, character analysis, and searchable information with precise timestamps as requested in the system instructions."""
    
    # Step 1: Get video files from GCS
    print("📁 Step 1: Getting video files from GCS...")
    video_files = get_gcs_video_files(BUCKET_NAME, VIDEO_FOLDER_PREFIX)
    
    if not video_files:
        print("❌ No video files found!")
        return None, None
    
    jsonl_documents = []
    
    # Step 2: Process each video
    for idx, blob in enumerate(video_files, 1):
        print(f"\n📹 Step 2: Processing video {idx}/{len(video_files)}: {blob.name}")
        
        # Create GCS URI
        file_uri = f"gs://{BUCKET_NAME}/{blob.name}"
        filename = blob.name.split('/')[-1]  # Get just the filename
        
        # Generate unique media ID
        media_id = str(uuid.uuid4())
        
        # Basic file information
        file_info = {
            'media_id': media_id,
            'filename': filename,
            'gcs_uri': file_uri,
            'file_size_mb': round(blob.size / (1024*1024), 2),
            'processing_date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
        }
        
        print(f"📊 File info: {filename} ({file_info['file_size_mb']} MB)")
        
        # Step 3: Filename analysis
        print("📝 Step 3: Analyzing filename...")
        content_analysis = analyze_filename_content(filename)
        
        # Step 4: Gemini video analysis (using your function)
        print("🤖 Step 4: Starting Gemini video analysis...")
        gemini_response = generate_video_analysis(system_instructions, main_prompt, file_uri)
        
        if gemini_response:
            try:
                # Parse JSON response from Gemini
                gemini_data = json.loads(gemini_response)
                gemini_analysis = gemini_data.get('response', 'No analysis provided')
                analysis_status = 'completed'
                print(f"✅ Gemini analysis successful")
            except json.JSONDecodeError as e:
                print(f"⚠️ JSON parsing issue, using raw response: {e}")
                gemini_analysis = gemini_response
                analysis_status = 'completed_with_parse_error'
        else:
            gemini_analysis = "Analysis failed - no response from Gemini"
            analysis_status = 'failed'
            print(f"❌ Gemini analysis failed")
        
        # Step 5: Create comprehensive text content
        print("📄 Step 5: Creating comprehensive text content...")
        
        comprehensive_text = f"""VIDEO ANALYSIS DOCUMENT
=========================

BASIC INFORMATION:
Filename: {filename}
Media ID: {media_id}
File Size: {file_info['file_size_mb']} MB
Processing Date: {file_info['processing_date']}
Original Video URI: {file_uri}

CONTENT CLASSIFICATION:
Genre: {content_analysis['genre']}
Content Type: {content_analysis['content_type']}
Language: {content_analysis['language']}
Category: {content_analysis['category']}

FILENAME-BASED METADATA:
{content_analysis['detailed_analysis']}

COMPREHENSIVE GEMINI VIDEO ANALYSIS:
===================================
{gemini_analysis}

PROCESSING STATUS:
Analysis Status: {analysis_status}
Processing Completed: {file_info['processing_date']}
"""
        
        # Step 6: Upload text file to GCS
        print("☁️ Step 6: Uploading text file to GCS...")
        
        # Create safe filename
        safe_filename = re.sub(r'[^a-zA-Z0-9_.-]', '_', filename)
        text_filename = f"{media_id}_{safe_filename}.txt"
        
        gcs_text_uri = upload_text_to_gcs(comprehensive_text, text_filename, BUCKET_NAME, TEXT_FILES_PREFIX)
        
        if gcs_text_uri:
            print(f"✅ Text file uploaded: {text_filename}")
            
            # Step 7: Extract structured metadata
            print("📋 Step 7: Extracting structured metadata...")
            
            extracted_metadata = extract_structured_metadata(filename, content_analysis)
            
            # Create structured data for JSONL
            struct_data = {
                "title": filename,
                "description": f"{content_analysis['content_type']} - {content_analysis['genre']}",
                "media_id": media_id,
                "genre": content_analysis['genre'],
                "content_type": content_analysis['content_type'],
                "language": content_analysis['language'],
                "category": content_analysis['category'],
                "year": extracted_metadata['year'],
                "file_size_mb": file_info['file_size_mb'],
                "analysis_status": analysis_status,
                "original_video_uri": file_uri,
                "processing_date": file_info['processing_date'],
                "keywords_en": content_analysis['keywords_en'],
                "keywords_id": content_analysis['keywords_id']
            }
            
            # Add optional structured fields
            if extracted_metadata['episode']:
                struct_data["episode"] = extracted_metadata['episode']
            if extracted_metadata['series_name']:
                struct_data["series_name"] = extracted_metadata['series_name']
            if extracted_metadata['characters']:
                struct_data["characters"] = extracted_metadata['characters']
            if extracted_metadata['teams']:
                struct_data["teams"] = extracted_metadata['teams']
            
            # Create JSONL document
            jsonl_doc = {
                "id": f"video-{idx:04d}",
                "structData": struct_data,
                "content": {
                    "mimeType": "text/plain",
                    "uri": gcs_text_uri
                }
            }
            
            jsonl_documents.append(jsonl_doc)
            print(f"✅ JSONL document created: video-{idx:04d}")
            
        else:
            print(f"❌ Failed to upload text file for {filename}")
        
        # Rate limiting between videos
        print("⏱️ Waiting 5 seconds before next video...")
        time.sleep(5)
    
    # Step 8: Write JSONL file
    print(f"\n📝 Step 8: Writing JSONL file...")
    
    with open(JSONL_OUTPUT_FILE, 'w', encoding='utf-8') as f:
        for doc in jsonl_documents:
            f.write(json.dumps(doc, ensure_ascii=False) + '\n')
    
    print(f"\n✅ COMPLETE PIPELINE FINISHED!")
    print(f"📊 SUMMARY:")
    print(f"  Videos processed: {len(jsonl_documents)}")
    print(f"  Text files created: gs://{BUCKET_NAME}/{TEXT_FILES_PREFIX}")
    print(f"  JSONL output: {JSONL_OUTPUT_FILE}")
    
    return jsonl_documents, JSONL_OUTPUT_FILE

def validate_pipeline_output(jsonl_file):
    """Validate the pipeline output."""
    
    print(f"🔍 VALIDATING PIPELINE OUTPUT: {jsonl_file}")
    print("=" * 50)
    
    try:
        with open(jsonl_file, 'r', encoding='utf-8') as f:
            lines = f.readlines()
        
        print(f"📄 Total documents: {len(lines)}")
        
        # Validate first few documents
        for i, line in enumerate(lines[:3]):
            doc = json.loads(line.strip())
            
            print(f"\n📋 Document {i+1}:")
            print(f"  ID: {doc.get('id')}")
            print(f"  Title: {doc.get('structData', {}).get('title')}")
            print(f"  Genre: {doc.get('structData', {}).get('genre')}")
            print(f"  Analysis Status: {doc.get('structData', {}).get('analysis_status')}")
            print(f"  Text File URI: {doc.get('content', {}).get('uri')}")
        
        print(f"\n✅ Validation complete!")
        
    except Exception as e:
        print(f"❌ Validation error: {e}")

# Main execution
if __name__ == "__main__":
    print("🎬 COMPLETE VIDEO TO JSONL PIPELINE READY!")
    print("\n🚀 To run the complete pipeline:")
    print("documents, jsonl_file = complete_video_processing_pipeline()")
    print("\n🔍 To validate output:")
    print("validate_pipeline_output('video_documents_scene.jsonl')")

🎬 COMPLETE VIDEO TO JSONL PIPELINE READY!

🚀 To run the complete pipeline:
documents, jsonl_file = complete_video_processing_pipeline()

🔍 To validate output:
validate_pipeline_output('video_documents_scene.jsonl')


## Run JSON creation

In [ ]:
# Run the complete pipeline
documents, jsonl_file = complete_video_processing_pipeline()

# Validate the output  
validate_pipeline_output('video_documents_scene.jsonl')

🚀 STARTING COMPLETE VIDEO PROCESSING PIPELINE
📁 Step 1: Getting video files from GCS...
🎬 Found 26 video files in GCS bucket

📹 Step 2: Processing video 1/26: video_analysis/Cinta Sedalam Rindu Episode 1 dan 2 - Gara Gara Bantuan Aluna, Galaxy Malah Dapat Sial.mp4
📊 File info: Cinta Sedalam Rindu Episode 1 dan 2 - Gara Gara Bantuan Aluna, Galaxy Malah Dapat Sial.mp4 (61.8 MB)
📝 Step 3: Analyzing filename...
🤖 Step 4: Starting Gemini video analysis...
🤖 Starting Gemini analysis for: gs://my-project-0004-346516-video-rag-data/video_analysis/Cinta Sedalam Rindu Episode 1 dan 2 - Gara Gara Bantuan Aluna, Galaxy Malah Dapat Sial.mp4
........................................ ✅ Analysis complete!
✅ Gemini analysis successful
📄 Step 5: Creating comprehensive text content...
☁️ Step 6: Uploading text file to GCS...
✅ Text file uploaded: de8718c5-b83b-4455-8660-5313381599d1_Cinta_Sedalam_Rindu_Episode_1_dan_2_-_Gara_Gara_Bantuan_Aluna__Galaxy_Malah_Dapat_Sial.mp4.txt
📋 Step 7: Extracting structur

In [ ]:
validate_pipeline_output('video_documents.jsonl')

### named datastore

In [ ]:
DATASTORE_NAME = "video-datastore-scene"
DATASTORE_ID = "video-id-scene_v2"
LOCATION = 'global'

# print variables for verification
print(f"Datastore name: {DATASTORE_NAME}")
print(f"Datastore ID: {DATASTORE_ID}")
!gsutil cp 'video_documents_scene.jsonl' 'gs://my-project-0004-346516-video-rag-data/'

### Create datastore

In [ ]:

# Helper Function to create data store
def create_data_store(
    project_id: str, location: str, data_store_name: str, data_store_id: str
):
    # Create a client
    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )
    client = discoveryengine.DataStoreServiceClient(client_options=client_options)

    # Initialize request argument(s)
    data_store = discoveryengine.DataStore(
        display_name=data_store_name,
        industry_vertical="GENERIC",
        content_config="CONTENT_REQUIRED",
    )

    request = discoveryengine.CreateDataStoreRequest(
        parent=discoveryengine.DataStoreServiceClient.collection_path(
            project_id, location, "default_collection"
        ),
        data_store=data_store,
        data_store_id=data_store_id,
    )
    operation = client.create_data_store(request=request)

    # Make the request
    # The try block is necessary to prevent execution from haulting due to an error being thrown when the datastore takes a while to instantiate
    try:
        response = operation.result(timeout=90)
    except:
        print("long-running operation")


In [ ]:
!pip install -q --upgrade google-cloud-discoveryengine


In [ ]:
from typing import Optional

from google.api_core.client_options import ClientOptions
from google.cloud import discoveryengine

# from google.cloud import discoveryengine_v1alpha as discoveryengine
from google.api_core.client_options import ClientOptions

import socket
import re

In [ ]:
# Create the datastore

create_data_store(PROJECT_ID, LOCATION, DATASTORE_NAME, DATASTORE_ID)
print(f"Datastore {DATASTORE_ID} successfully created")


In [ ]:
source_documents = 'gs://my-project-0004-346516-video-rag-data'

In [ ]:
# Helper Function to import documents from GCS bucket into datastore
def import_documents(
    project_id: str,
    location: str,
    data_store_id: str,
    gcs_uri: str,
):
    # Create a client
    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )
    client = discoveryengine.DocumentServiceClient(client_options=client_options)

    # The full resource name of the search engine branch.
    # e.g. projects/{project}/locations/{location}/dataStores/{data_store_id}/branches/{branch}
    parent = client.branch_path(
        project=project_id,
        location=location,
        data_store=data_store_id,
        branch="default_branch",
    )

    source_documents =  [f"{gcs_uri}/*"]

    request = discoveryengine.ImportDocumentsRequest(
        parent=parent,
        gcs_source=discoveryengine.GcsSource(
            input_uris=source_documents, data_schema="document"
        ),
        # Options: `FULL`, `INCREMENTAL`
        reconciliation_mode=discoveryengine.ImportDocumentsRequest.ReconciliationMode.INCREMENTAL,
    )

    # Make the request
    operation = client.import_documents(request=request)

    response = operation.result()

    # Once the operation is complete,
    # get information from operation metadata
    metadata = discoveryengine.ImportDocumentsMetadata(operation.metadata)

    # Handle the response
    return operation.operation.name

In [ ]:
import_documents(PROJECT_ID, LOCATION, DATASTORE_ID, source_documents)


'projects/255766800726/locations/global/collections/default_collection/dataStores/video-id-scene_v2/branches/0/operations/import-documents-17401627510891150077'

### Create search app

In [ ]:
# Helper function to create a Vertex Search Engine
def create_engine(
    project_id: str, location: str, data_store_name: str, data_store_id: str
):
    # Create a client
    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )
    client = discoveryengine.EngineServiceClient(client_options=client_options)

    # Initialize request argument(s)
    config = discoveryengine.Engine.SearchEngineConfig(
        search_tier="SEARCH_TIER_ENTERPRISE", search_add_ons=["SEARCH_ADD_ON_LLM"]
    )

    engine = discoveryengine.Engine(
        display_name=data_store_name,
        solution_type="SOLUTION_TYPE_SEARCH",
        industry_vertical="GENERIC",
        data_store_ids=[data_store_id],
        search_engine_config=config,
    )

    request = discoveryengine.CreateEngineRequest(
        parent=discoveryengine.DataStoreServiceClient.collection_path(
            project_id, location, "default_collection"
        ),
        engine=engine,
        engine_id=engine.display_name,
    )

    # Make the request
    operation = client.create_engine(request=request)
    response = operation.result(timeout=90)


In [ ]:
# Create the Vertex Search Engine
# DATASTORE_NAME = 'video_datastore_2manual'
# DATASTORE_ID = 'video-datastore-2manual_1758643372913' 
try:
    create_engine(PROJECT_ID, LOCATION, DATASTORE_NAME, DATASTORE_ID)
except:
    print("if not running first time, create_engine may already exist")

### Datastore Query

In [30]:
from typing import List

def search_sample_v1(
    project_id: str,
    location: str,
    data_store_id: str,
    search_query: str,
) -> List[discoveryengine.SearchResponse]:
    #  For more information, refer to:
    # https://cloud.google.com/generative-ai-app-builder/docs/locations#specify_a_multi-region_for_your_data_store
    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if LOCATION != "global"
        else None
    )

    # Create a client
    client = discoveryengine.SearchServiceClient(client_options=client_options)

    # The full resource name of the search engine serving config
    # e.g. projects/{project_id}/locations/{location}/dataStores/{data_store_id}/servingConfigs/{serving_config_id}
    serving_config = client.serving_config_path(
        project=project_id,
        location=location,
        data_store=data_store_id,
        serving_config="default_config",
    )

    # Optional: Configuration options for search
    # Refer to the `ContentSearchSpec` reference for all supported fields:
    # https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.types.SearchRequest.ContentSearchSpec
    content_search_spec = discoveryengine.SearchRequest.ContentSearchSpec(
        # For information about snippets, refer to:
        # https://cloud.google.com/generative-ai-app-builder/docs/snippets
        snippet_spec=discoveryengine.SearchRequest.ContentSearchSpec.SnippetSpec(
            return_snippet=True
        ),
        # For information about search summaries, refer to:
        # https://cloud.google.com/generative-ai-app-builder/docs/get-search-summaries
        summary_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec(
            summary_result_count=5,
            include_citations=True,
            ignore_adversarial_query=True,
            ignore_non_summary_seeking_query=False,
        ),
    )

    # Refer to the `SearchRequest` reference for all supported fields:
    # https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.types.SearchRequest
    request = discoveryengine.SearchRequest(
        serving_config=serving_config,
        query=search_query,
        page_size=10,
        content_search_spec=content_search_spec,
        query_expansion_spec=discoveryengine.SearchRequest.QueryExpansionSpec(
            condition=discoveryengine.SearchRequest.QueryExpansionSpec.Condition.AUTO,
        ),
        spell_correction_spec=discoveryengine.SearchRequest.SpellCorrectionSpec(
            mode=discoveryengine.SearchRequest.SpellCorrectionSpec.Mode.AUTO
        ),
    )

    response = client.search(request)
    return response

In [31]:
# Ask a sample query to get an answer from the search engine!
query = "Aluna fall from horse?"

print(search_sample_v1(PROJECT_ID, LOCATION, DATASTORE_ID, query).summary)

summary_text: "Aluna was pulled off her feet by a horse\'s momentum and onto its back [1]. Aluna was trying to help and apologized for the accident [2]. Someone asked, \"How could you do this to me and Aluna?\" [4]. Aluna performed the Heimlich maneuver on Jovita, who then fell face-first into a bag of tapioca flour [5]. Aluna and Ezra bumped foreheads and stumbled, ending up in a close position [3].\n"
summary_with_metadata {
  summary: "Aluna was pulled off her feet by a horse\'s momentum and onto its back. Aluna was trying to help and apologized for the accident. Someone asked, \"How could you do this to me and Aluna?\". Aluna performed the Heimlich maneuver on Jovita, who then fell face-first into a bag of tapioca flour. Aluna and Ezra bumped foreheads and stumbled, ending up in a close position.\n"
  citation_metadata {
    citations {
      end_index: 70
      sources {
      }
    }
    citations {
      start_index: 71
      end_index: 128
      sources {
        reference_inde

In [32]:
SEARCH_QUERY = "Aluna fall from horse?"
VERTEX_AI_SEARCH_APP_ID = 'video_datastore_2manual'

CUSTOM_PROMPT = """
      You are a helpful assistant knowledgeable about data.
      Be mindful of the time frame user inputs
      Help user with their queries related with data, it can be anything
"""

In [33]:
def search_spec():
    content_search_spec = discoveryengine.SearchRequest.ContentSearchSpec(
        snippet_spec=discoveryengine.SearchRequest.ContentSearchSpec.SnippetSpec(
            return_snippet=True
        ),
        summary_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec(
            summary_result_count=10,
            include_citations=True,
            ignore_adversarial_query=True,
            ignore_non_summary_seeking_query=True,
            model_prompt_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec.ModelPromptSpec(
                preamble=CUSTOM_PROMPT
            ),
            model_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec.ModelSpec(
                version="stable",
            ),
        ),
    )
    return content_search_spec


def search_sample(project_id: str, location: str, engine_id: str, search_query: str):
    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )

    client = discoveryengine.SearchServiceClient(client_options=client_options)

    serving_config = f"projects/{project_id}/locations/{location}/collections/default_collection/engines/{engine_id}/servingConfigs/default_config"

    content_search_spec = search_spec()

    request = discoveryengine.SearchRequest(
        serving_config=serving_config,
        query=search_query,
        page_size=10,
        content_search_spec=content_search_spec,
        query_expansion_spec=discoveryengine.SearchRequest.QueryExpansionSpec(
            condition=discoveryengine.SearchRequest.QueryExpansionSpec.Condition.AUTO,
        ),
        spell_correction_spec=discoveryengine.SearchRequest.SpellCorrectionSpec(
            mode=discoveryengine.SearchRequest.SpellCorrectionSpec.Mode.AUTO
        ),
    )
    response = client.search(request)

    return response



In [34]:

search_response = search_sample(
    PROJECT_ID, 'global', VERTEX_AI_SEARCH_APP_ID, SEARCH_QUERY
)
search_response.summary.summary_with_metadata

summary: "Yes, Aluna does fall from a horse. She profusely apologizes for causing the fall and feels very guilty about it. Galaksi, who was with her, notices his horse, \"Boy,\" has run away and then realizes his wristwatch is missing, causing him to panic and search for it. Earlier, Galaksi had told Aluna to hold on to him while riding to prevent her from falling."
citation_metadata {
  citations {
    end_index: 34
    sources {
    }
  }
  citations {
    start_index: 35
    end_index: 112
    sources {
    }
  }
  citations {
    start_index: 113
    end_index: 263
    sources {
    }
  }
  citations {
    start_index: 264
    end_index: 355
    sources {
      reference_index: 2
    }
  }
}
references {
  title: "Cinta Sedalam Rindu Episode 1 dan 2 - Gara Gara Bantuan Aluna, Galaxy Malah Dapat Sial.mp4"
  document: "projects/255766800726/locations/global/collections/default_collection/dataStores/video-datastore-2manual_1758643372913/branches/0/documents/video-0001"
}
references {


### Create video details

In [ ]:
# Load Golden Set for Evaluation
import pandas as pd

try:
    golden_set = pd.read_csv("GoldenSet.csv")
    print(f"✅ Golden Set loaded successfully!")
    print(f"📊 Total test queries: {len(golden_set)}")
    print(f"📋 Query types: {golden_set['query_type'].unique()}")
    print(f"📈 Difficulty levels: {golden_set['difficulty'].unique()}")
    print(f"\n🔍 Sample query:")
    print(f"  EN: {golden_set.iloc[0]['query_en']}")
    print(f"  ID: {golden_set.iloc[0]['query_id']}")
except FileNotFoundError:
    print("⚠️ GoldenSet.csv not found. Evaluation will be skipped.")
    golden_set = None
except Exception as e:
    print(f"❌ Error loading Golden Set: {e}")
    golden_set = None

✅ Golden Set loaded successfully!
📊 Total test queries: 30
📋 Query types: ['Soap opera' 'News' 'Sports']
📈 Difficulty levels: ['Easy' 'Medium']

🔍 Sample query:
  EN: Aluna fall from horse
  ID: Aluna jatuh dari kuda


### NEW GOLDEN TEST

In [ ]:
# Load Golden Set for Discovery Engine Evaluation
import pandas as pd
from google.api_core.client_options import ClientOptions
from google.cloud import discoveryengine
import time

# Discovery Engine Configuration
PROJECT_ID = "my-project-0004-346516"  # Update with your project ID
# VERTEX_AI_SEARCH_APP_ID = 'video_datastore_2manual'
LOCATION = 'global'
CUSTOM_PROMPT = """
You are an expert video content analyst specializing in Indonesian entertainment, sports, and news content. Your primary goal is to provide specific, detailed, and accurate information from video sources.

**For News Content:**
- Extract and provide specific details like: addresses, house numbers, building names, exact locations
- Identify all people mentioned by name, their roles, and what they say or do
- Note specific events, incidents, and their precise timing within videos
- Include quotes, statements, interviews, and conversations
- Mention visual elements: banners, signs, buildings, vehicles, clothing
- Provide context about locations, investigations, demonstrations, or official statements

**For Sports Content:**
- Identify players by name, team, position, and specific actions (goals, fouls, cards)
- Note exact timing of events, scores, and match developments
- Mention referees, officials, and their decisions

**For Drama/Entertainment:**
- Name characters and actors, their relationships and interactions
- Describe specific scenes, dialogue, and plot developments with timing
- Include emotional moments, conflicts, and resolutions

**General Instructions:**
- Always provide timestamps when events occur in videos
- Be specific about WHO does WHAT and WHEN it happens
- If asked about specific details (like house numbers, addresses, quotes), search thoroughly through all available content
- Don't say "no information available" if there are related details - provide whatever relevant information exists
- Connect user queries to similar or related content even if not exact matches
- Focus on factual, verifiable details from the video sources

Remember: Users need specific details for research and reference purposes. Provide comprehensive information even for seemingly minor details."""


# Your Discovery Engine search functions
def search_spec():
    content_search_spec = discoveryengine.SearchRequest.ContentSearchSpec(
        snippet_spec=discoveryengine.SearchRequest.ContentSearchSpec.SnippetSpec(
            return_snippet=True
        ),
        summary_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec(
            summary_result_count=10,
            include_citations=True,
            ignore_adversarial_query=True,
            ignore_non_summary_seeking_query=True,
            model_prompt_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec.ModelPromptSpec(
                preamble=CUSTOM_PROMPT
            ),
            model_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec.ModelSpec(
                version="stable",
            ),
        ),
    )
    return content_search_spec

def search_sample(project_id: str, location: str, engine_id: str, search_query: str):
    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )
    client = discoveryengine.SearchServiceClient(client_options=client_options)
    serving_config = f"projects/{project_id}/locations/{location}/collections/default_collection/engines/{engine_id}/servingConfigs/default_config"
    content_search_spec = search_spec()
    request = discoveryengine.SearchRequest(
        serving_config=serving_config,
        query=search_query,
        page_size=10,
        content_search_spec=content_search_spec,
        query_expansion_spec=discoveryengine.SearchRequest.QueryExpansionSpec(
            condition=discoveryengine.SearchRequest.QueryExpansionSpec.Condition.AUTO,
        ),
        spell_correction_spec=discoveryengine.SearchRequest.SpellCorrectionSpec(
            mode=discoveryengine.SearchRequest.SpellCorrectionSpec.Mode.AUTO
        ),
    )
    response = client.search(request)
    return response

def test_golden_set_with_discovery_engine(golden_set_df, num_tests=None, query_types=None):
    """
    Test Discovery Engine system with golden dataset.
    """
    
    if golden_set_df is None:
        print("❌ No golden set available for testing")
        return
    
    print("🧪 GOLDEN SET EVALUATION WITH DISCOVERY ENGINE")
    print("=" * 80)
    
    # Filter by query types if specified
    test_df = golden_set_df.copy()
    if query_types:
        test_df = test_df[test_df['query_type'].isin(query_types)]
        print(f"📋 Filtering for query types: {query_types}")
    
    # Limit number of tests if specified
    if num_tests:
        test_df = test_df.head(num_tests)
        print(f"📊 Testing first {num_tests} queries")
    
    print(f"\n🎯 Running {len(test_df)} evaluation queries...\n")
    
    results = []
    
    for idx, row in test_df.iterrows():
        query_id = row.get('id', f"Q-{idx:03d}")
        query_en = row.get('query_en', '')
        query_id_lang = row.get('query_id', '')  # Indonesian query
        query_type = row.get('query_type', 'Unknown')
        expected_answer = row.get('expected_answer', '')
        difficulty = row.get('difficulty', 'Unknown')
        
        print(f"\n🔍 {query_id} - {query_type} ({difficulty})")
        print(f"📝 Query (EN): {query_en}")
        if query_id_lang:
            print(f"📝 Query (ID): {query_id_lang}")
        print(f"🎯 Expected: {expected_answer}")
        print("-" * 60)
        
        try:
            start_time = time.time()
            
            # Use Discovery Engine search
            print("🔍 Searching with Discovery Engine...")
            
            search_response = search_sample(
                PROJECT_ID, LOCATION, VERTEX_AI_SEARCH_APP_ID, query_en
            )
            
            response_time = time.time() - start_time
            
            # Extract response and sources
            if hasattr(search_response, 'summary') and search_response.summary:
                summary_response = search_response.summary.summary_with_metadata
                discovery_response = summary_response.summary if summary_response else "No summary available"
                
                # Extract references/sources
                references = []
                if summary_response and hasattr(summary_response, 'references'):
                    references = [ref.title for ref in summary_response.references]
                
                num_sources = len(references)
                top_source = references[0] if references else None
                
            else:
                discovery_response = "No response from Discovery Engine"
                references = []
                num_sources = 0
                top_source = None
            
            print(f"🤖 Discovery Engine Response ({response_time:.2f}s):")
            print(discovery_response)
            
            # Analyze response quality for Discovery Engine
            quality_analysis = analyze_discovery_response_quality(discovery_response, expected_answer)
            
            # Quality status based on Discovery Engine response
            if not quality_analysis['apologetic'] and quality_analysis['specific_content'] and num_sources > 0:
                quality_status = "✅ EXCELLENT"
            elif not quality_analysis['apologetic'] and quality_analysis['found_information']:
                quality_status = "✅ GOOD"
            elif quality_analysis['helpful'] and not quality_analysis['apologetic']:
                quality_status = "⚠️ PARTIAL"
            else:
                quality_status = "❌ POOR"
            
            print(f"\n📊 Response Quality: {quality_status}")
            
            # Show quality indicators
            if quality_analysis['apologetic']:
                print("   - ⚠️ Response was apologetic or lacking info")
            if quality_analysis['found_information']:
                print("   - ✅ Found relevant information")
            if quality_analysis['specific_content']:
                print("   - ✅ Contains specific content details")
            if quality_analysis['mentions_video']:
                print("   - ✅ Mentions video/episode information")
            if quality_analysis['has_timestamp']:
                print("   - ✅ Contains timestamp information")
            
            print(f"\n📄 Source References: {num_sources} sources found")
            if num_sources > 0:
                for i, ref_title in enumerate(references[:3], 1):
                    print(f"  {i}. {ref_title}")
            else:
                print("  ⚠️ No source references found")
            
            # Calculate quality score for Discovery Engine
            quality_score = calculate_discovery_quality_score(quality_analysis, num_sources)
            
            # Store results
            results.append({
                'query_id': query_id,
                'query_en': query_en,
                'query_type': query_type,
                'difficulty': difficulty,
                'expected_answer': expected_answer,
                'discovery_response': discovery_response,
                'response_time': response_time,
                'num_sources': num_sources,
                'top_source': top_source,
                'source_references': references,
                'quality_status': quality_status,
                'quality_score': quality_score,
                'found_information': quality_analysis['found_information'],
                'apologetic': quality_analysis['apologetic'],
                'specific_content': quality_analysis['specific_content'],
                'mentions_video': quality_analysis['mentions_video'],
                'has_timestamp': quality_analysis['has_timestamp']
            })
            
        except Exception as e:
            print(f"❌ Error processing query: {e}")
            results.append({
                'query_id': query_id,
                'query_en': query_en,
                'query_type': query_type,
                'difficulty': difficulty,
                'expected_answer': expected_answer,
                'discovery_response': f"ERROR: {e}",
                'response_time': 0,
                'num_sources': 0,
                'top_source': None,
                'source_references': [],
                'quality_status': "❌ ERROR",
                'quality_score': 0,
                'found_information': False,
                'apologetic': True,
                'specific_content': False,
                'mentions_video': False,
                'has_timestamp': False
            })
        
        print("\n" + "=" * 80)
        time.sleep(1)  # Brief pause between queries
    
    # Results summary
    results_df = pd.DataFrame(results)
    
    print(f"\n📊 DISCOVERY ENGINE EVALUATION SUMMARY")
    print("=" * 60)
    print(f"Total Queries Tested: {len(results_df)}")
    
    # Quality breakdown
    excellent = len(results_df[results_df['quality_status'] == '✅ EXCELLENT'])
    good = len(results_df[results_df['quality_status'] == '✅ GOOD'])
    partial = len(results_df[results_df['quality_status'] == '⚠️ PARTIAL'])
    poor = len(results_df[results_df['quality_status'] == '❌ POOR'])
    errors = len(results_df[results_df['quality_status'] == '❌ ERROR'])
    
    print(f"\nQuality Distribution:")
    print(f"  Excellent: {excellent}/{len(results_df)} ({(excellent/len(results_df)*100):.1f}%)")
    print(f"  Good: {good}/{len(results_df)} ({(good/len(results_df)*100):.1f}%)")
    print(f"  Partial: {partial}/{len(results_df)} ({(partial/len(results_df)*100):.1f}%)")
    print(f"  Poor: {poor}/{len(results_df)} ({(poor/len(results_df)*100):.1f}%)")
    if errors > 0:
        print(f"  Errors: {errors}/{len(results_df)} ({(errors/len(results_df)*100):.1f}%)")
    
    print(f"\nPerformance Metrics:")
    print(f"  Average quality score: {results_df['quality_score'].mean():.1f}/10")
    print(f"  Average response time: {results_df['response_time'].mean():.2f}s")
    
    # Source retrieval stats
    with_sources = len(results_df[results_df['num_sources'] > 0])
    print(f"  Queries with sources: {with_sources}/{len(results_df)} ({(with_sources/len(results_df)*100):.1f}%)")
    
    # Query type breakdown
    print(f"\nPerformance by Query Type:")
    for qtype in results_df['query_type'].unique():
        type_results = results_df[results_df['query_type'] == qtype]
        avg_score = type_results['quality_score'].mean()
        success_rate = len(type_results[type_results['found_information']]) / len(type_results) * 100
        print(f"  {qtype}: Avg Score {avg_score:.1f}, Success Rate {success_rate:.1f}%")
    
    # Show best and worst performing queries
    print(f"\n🏆 Best Performing Queries:")
    top_queries = results_df.nlargest(3, 'quality_score')[['query_id', 'query_en', 'quality_score', 'quality_status']]
    for _, query in top_queries.iterrows():
        print(f"  {query['query_id']}: {query['query_en'][:50]}... (Score: {query['quality_score']:.1f})")
    
    print(f"\n⚠️ Worst Performing Queries:")
    worst_queries = results_df.nsmallest(3, 'quality_score')[['query_id', 'query_en', 'quality_score', 'quality_status']]
    for _, query in worst_queries.iterrows():
        print(f"  {query['query_id']}: {query['query_en'][:50]}... (Score: {query['quality_score']:.1f})")
    
    return results_df

def analyze_discovery_response_quality(response_text, expected_answer):
    """Analyze the quality of a Discovery Engine response."""
    
    response_lower = response_text.lower()
    
    # Check for no information or errors
    no_info_phrases = [
        "no summary available", "no response", "no information", 
        "not found", "cannot find", "unable to", "don't have information"
    ]
    apologetic = any(phrase in response_lower for phrase in no_info_phrases)
    
    # Check if response contains useful information
    found_information = len(response_text.strip()) > 20 and not apologetic
    
    # Check for specific content mentions (enhanced for video content)
    specific_indicators = [
        "episode", "scene", "character", "dialogue", "video", "minute", 
        "second", "timestamp", "around", "at", "galaksi", "aluna", "galaxy"
    ]
    specific_content = any(indicator in response_lower for indicator in specific_indicators)
    
    # Check if response mentions videos/episodes
    video_indicators = ["video", "episode", "series", "film", "movie", "content", "mp4"]
    mentions_video = any(indicator in response_lower for indicator in video_indicators)
    
    # Check for timestamp information
    timestamp_indicators = ["around", "at", "minute", "second", ":", "time"]
    has_timestamp = any(indicator in response_lower for indicator in timestamp_indicators)
    
    # General helpfulness
    helpful = not apologetic and len(response_text.strip()) > 10
    
    return {
        'apologetic': apologetic,
        'found_information': found_information,
        'helpful': helpful,
        'specific_content': specific_content,
        'mentions_video': mentions_video,
        'has_timestamp': has_timestamp,
        'response_length': len(response_text)
    }

def calculate_discovery_quality_score(quality_analysis, num_sources):
    """Calculate quality score from 0-10 for Discovery Engine responses."""
    
    score = 0
    
    # Base scoring
    if not quality_analysis['apologetic']:
        score += 2
    
    if quality_analysis['helpful']:
        score += 1
    
    if quality_analysis['found_information']:
        score += 3
    
    if quality_analysis['specific_content']:
        score += 2
    
    if quality_analysis['mentions_video']:
        score += 1
    
    if quality_analysis['has_timestamp']:
        score += 1  # Bonus for timestamp info
    
    # Bonus for source references (Discovery Engine specific)
    if num_sources > 0:
        score += min(num_sources * 0.5, 2)  # Up to 2 extra points for sources
    
    return min(score, 10)

# Load Golden Set for Discovery Engine Evaluation
try:
    golden_set = pd.read_csv("GoldenSet.csv")
    print(f"✅ Golden Set loaded successfully!")
    print(f"📊 Total test queries: {len(golden_set)}")
    print(f"📋 Query types: {golden_set['query_type'].unique()}")
    print(f"📈 Difficulty levels: {golden_set['difficulty'].unique()}")
    print(f"\n🔍 Sample query:")
    print(f"  EN: {golden_set.iloc[0]['query_en']}")
    print(f"  ID: {golden_set.iloc[0]['query_id']}")
    print(f"\n🚀 Discovery Engine evaluation ready!")
    print(f"💡 Run: test_golden_set_with_discovery_engine(golden_set, num_tests=5)")
except FileNotFoundError:
    print("⚠️ GoldenSet.csv not found. Evaluation will be skipped.")
    golden_set = None
except Exception as e:
    print(f"❌ Error loading Golden Set: {e}")
    golden_set = None

# Quick test function
def quick_test_discovery_engine():
    """Quick test with first 5 queries using Discovery Engine."""
    if golden_set is not None:
        return test_golden_set_with_discovery_engine(golden_set, num_tests=30)
    else:
        print("❌ Golden set not available")
        return None

print("\n🧪 DISCOVERY ENGINE GOLDEN SET EVALUATION READY!")
print("\n💡 Available functions:")
print("test_golden_set_with_discovery_engine(golden_set, num_tests=5)  # Test first 5")
print("test_golden_set_with_discovery_engine(golden_set)  # Test all")
print("test_golden_set_with_discovery_engine(golden_set, query_types=['Sports'])  # Filter by type")
print("quick_test_discovery_engine()  # Quick 5-query test")

✅ Golden Set loaded successfully!
📊 Total test queries: 30
📋 Query types: ['Soap opera' 'News' 'Sports']
📈 Difficulty levels: ['Easy' 'Medium']

🔍 Sample query:
  EN: Aluna fall from horse
  ID: Aluna jatuh dari kuda

🚀 Discovery Engine evaluation ready!
💡 Run: test_golden_set_with_discovery_engine(golden_set, num_tests=5)

🧪 DISCOVERY ENGINE GOLDEN SET EVALUATION READY!

💡 Available functions:
test_golden_set_with_discovery_engine(golden_set, num_tests=5)  # Test first 5
test_golden_set_with_discovery_engine(golden_set)  # Test all
test_golden_set_with_discovery_engine(golden_set, query_types=['Sports'])  # Filter by type
quick_test_discovery_engine()  # Quick 5-query test


In [ ]:
quick_test_discovery_engine()

🧪 GOLDEN SET EVALUATION WITH DISCOVERY ENGINE
📊 Testing first 30 queries

🎯 Running 30 evaluation queries...


🔍 Q-001 - Soap opera (Easy)
📝 Query (EN): Aluna fall from horse
📝 Query (ID): Aluna jatuh dari kuda
🎯 Expected: Episode 1&2 - around 00:54
------------------------------------------------------------
🔍 Searching with Discovery Engine...
🤖 Discovery Engine Response (3.12s):
Aluna falls from a horse in "Cinta Sedalam Rindu Episode 1 dan 2 - Gara Gara Bantuan Aluna, Galaxy Malah Dapat Sial.mp4". Aluna profusely apologizes for causing the fall, feeling very guilty. Galaksi, who was riding the horse with Aluna, notices the horse, which he calls 'Boy', has run away. He then realizes his wristwatch is missing and begins to panic, searching for it in the tall grass between 02:49 and 03:12. Earlier, at 01:32 in "Cinta Sedalam Rindu Episode 4 dan 5 - Ihiy! Galaxy Ngarep Pacaran Sama Aluna.mp4", Galaksi tells Aluna to "Hold on. You'll fall later" while riding.

📊 Response Quality: ✅ EXCE

,query_id,query_en,query_type,difficulty,expected_answer,discovery_response,response_time,num_sources,top_source,source_references,quality_status,quality_score,found_information,apologetic,specific_content,mentions_video,has_timestamp
0,Q-001,Aluna fall from horse,Soap opera,Easy,Episode 1&2 - around 00:54,"Aluna falls from a horse in ""Cinta Sedalam Rin...",3.117598,8,Cinta Sedalam Rindu Episode 1 dan 2 - Gara Gar...,[Cinta Sedalam Rindu Episode 1 dan 2 - Gara Ga...,✅ EXCELLENT,10.0,True,False,True,True,True
1,Q-002,Aluna step on watch,Soap opera,Medium,Episode 1&2 - around 03:45,No summary available,1.222317,0,None,[],❌ POOR,0.0,False,True,False,False,False
2,Q-003,Galaxy ride a sports car,Soap opera,Easy,Episode 3A - around 01:35\nEpisode 3B - around...,"In the drama ""Cinta Sedalam Rindu,"" the charac...",2.592232,2,Cinta Sedalam Rindu Episode 3 - Salting Abis!!...,[Cinta Sedalam Rindu Episode 3 - Salting Abis!...,✅ EXCELLENT,9.0,True,False,True,False,False
3,Q-004,Aluna meet her father,Soap opera,Easy,Episode 4&5A - around 00:25,Aluna meets her father in a scene where she he...,2.745677,8,Cinta Sedalam Rindu Episode 4 dan 5 - BAPER!!!...,[Cinta Sedalam Rindu Episode 4 dan 5 - BAPER!!...,✅ EXCELLENT,10.0,True,False,True,False,True
4,Q-005,Aluna's mother mad at her husband,Soap opera,Easy,Episode 4&5A - around 01:33,"Aluna's mother, Novia, is seen in a light blue...",3.192566,9,Cinta Sedalam Rindu Episode 4 dan 5 - BAPER!!!...,[Cinta Sedalam Rindu Episode 4 dan 5 - BAPER!!...,❌ POOR,5.0,False,True,True,False,True
5,Q-006,Galaxy fetches Aluna with motorbike,Soap opera,Easy,Episode 4&5B - around 00:20,"At 00:03 in ""Cinta Sedalam Rindu Episode 4 dan...",3.456753,6,Cinta Sedalam Rindu Episode 4 dan 5 - Ihiy! Ga...,[Cinta Sedalam Rindu Episode 4 dan 5 - Ihiy! G...,✅ EXCELLENT,10.0,True,False,True,True,True
6,Q-007,Aluna suspects that Galaxy is not a horse groom,Soap opera,Medium,Episode 6A - around 02:00,"Yes, Aluna suspects that Galaxy (also referred...",4.586049,8,Cinta Sedalam Rindu Episode 1 dan 2 - Gara Gar...,[Cinta Sedalam Rindu Episode 1 dan 2 - Gara Ga...,✅ EXCELLENT,10.0,True,False,True,False,True
7,Q-008,Robby pulls Galaxy's ear,Soap opera,Easy,Episode 6A - around 05:54,No summary available,1.064703,0,None,[],❌ POOR,0.0,False,True,False,False,False
8,Q-009,Mr. Omar was caught cheating by his wife,Soap opera,Medium,Episode 6B - around 00:35,No summary available,1.614536,0,None,[],❌ POOR,0.0,False,True,False,False,False
9,Q-010,Ezra taking photo of Felicia,Soap opera,Easy,Episode 6B - around 06:10,"In ""Cinta Sedalam Rindu Episode 6 - Ngakak! Ez...",3.107157,4,Cinta Sedalam Rindu Episode 6 - Ngakak! Ezra K...,[Cinta Sedalam Rindu Episode 6 - Ngakak! Ezra ...,✅ EXCELLENT,10.0,True,False,True,True,True


### Snippet 

In [ ]:
from google.api_core.client_options import ClientOptions
from google.cloud import discoveryengine_v1 as discoveryengine

# TODO(developer): Uncomment these variables before running the sample.
# project_id = "YOUR_PROJECT_ID"
# location = "YOUR_LOCATION"          # Values: "global", "us", "eu"
engine_id = VERTEX_AI_SEARCH_APP_ID
search_query = "Trump was asked whether the killer was a hitman"


def search_sample(
    project_id: str,
    location: str,
    engine_id: str,
    search_query: str,
) -> discoveryengine.services.search_service.pagers.SearchPager:
    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )

    # Create a client
    client = discoveryengine.SearchServiceClient(client_options=client_options)

    # The full resource name of the search app serving config
    serving_config = f"projects/{project_id}/locations/{location}/collections/default_collection/engines/{engine_id}/servingConfigs/default_config"

    content_search_spec = discoveryengine.SearchRequest.ContentSearchSpec(
        # For information about snippets, refer to: https://cloud.google.com/generative-ai-app-builder/docs/snippets
        snippet_spec=discoveryengine.SearchRequest.ContentSearchSpec.SnippetSpec(
            return_snippet=True
        ),
        # For information about search summaries, refer to:
        # https://cloud.google.com/generative-ai-app-builder/docs/get-search-summaries
        summary_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec(
            summary_result_count=5,
            include_citations=True,
            ignore_adversarial_query=True,
            ignore_non_summary_seeking_query=True,
            model_prompt_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec.ModelPromptSpec(
                preamble="YOUR_CUSTOM_PROMPT"
            ),
            model_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec.ModelSpec(
                version="stable",
            ),
        ),
    )

    # Refer to the `SearchRequest` reference for all supported fields:
    # https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.types.SearchRequest
    request = discoveryengine.SearchRequest(
        serving_config=serving_config,
        query=search_query,
        page_size=10,
        content_search_spec=content_search_spec,
        query_expansion_spec=discoveryengine.SearchRequest.QueryExpansionSpec(
            condition=discoveryengine.SearchRequest.QueryExpansionSpec.Condition.AUTO,
        ),
        spell_correction_spec=discoveryengine.SearchRequest.SpellCorrectionSpec(
            mode=discoveryengine.SearchRequest.SpellCorrectionSpec.Mode.AUTO
        ),
        # Optional: Use fine-tuned model for this request
        # custom_fine_tuning_spec=discoveryengine.CustomFineTuningSpec(
        #     enable_search_adaptor=True
        # ),
    )

    page_result = client.search(request)

    # Handle the response
    for response in page_result:
        print(response)

    return page_result


In [ ]:
search_sample(PROJECT_ID, LOCATION, VERTEX_AI_SEARCH_APP_ID, search_query)

id: "video-0025"
document {
  name: "projects/255766800726/locations/global/collections/default_collection/dataStores/video-datastore-2manual_1758643372913/branches/0/documents/video-0025"
  id: "video-0025"
  struct_data {
    fields {
      key: "analysis_status"
      value {
        string_value: "completed"
      }
    }
    fields {
      key: "category"
      value {
        string_value: "News & Current Affairs"
      }
    }
    fields {
      key: "content_type"
      value {
        string_value: "News Report"
      }
    }
    fields {
      key: "file_size_mb"
      value {
        number_value: 54.26
      }
    }
    fields {
      key: "genre"
      value {
        string_value: "News/Journalism"
      }
    }
    fields {
      key: "keywords_en"
      value {
        list_value {
          values {
            string_value: "news"
          }
          values {
            string_value: "journalism"
          }
          values {
            string_value: "report"
   

SearchPager<results {
  id: "video-0025"
  document {
    name: "projects/255766800726/locations/global/collections/default_collection/dataStores/video-datastore-2manual_1758643372913/branches/0/documents/video-0025"
    id: "video-0025"
    struct_data {
      fields {
        key: "analysis_status"
        value {
          string_value: "completed"
        }
      }
      fields {
        key: "category"
        value {
          string_value: "News & Current Affairs"
        }
      }
      fields {
        key: "content_type"
        value {
          string_value: "News Report"
        }
      }
      fields {
        key: "file_size_mb"
        value {
          number_value: 54.26
        }
      }
      fields {
        key: "genre"
        value {
          string_value: "News/Journalism"
        }
      }
      fields {
        key: "keywords_en"
        value {
          list_value {
            values {
              string_value: "news"
            }
            values {


### 

In [ ]:
!pip install -q -U langchain_google_community

In [ ]:
from langchain_google_community import (
    VertexAIMultiTurnSearchRetriever,
    VertexAISearchRetriever,
)

SEARCH_ENGINE_ID =VERTEX_AI_SEARCH_APP_ID  # Set to your search app ID
DATA_STORE_ID = DATASTORE_ID #DATASTORE_NAME # DATASTORE_ID


retriever = VertexAISearchRetriever(
    project_id=PROJECT_ID,
    location_id=LOCATION,
    data_store_id=DATA_STORE_ID,
    max_documents=3,
)

/opt/conda/lib/python3.10/site-packages/langchain_google_community/vertex_ai_search.py:365: UserWarning: Beta features are configured but beta=False. The following beta features will be ignored:['custom_embedding_ratio']
  warnings.warn(


In [ ]:
search_query #= "What are Alphabet's Other Bets?"

result = retriever.invoke(search_query)
for doc in result:
    print(doc)

FailedPrecondition: 400 Cannot use enterprise edition features (website search, multi-modal search, extractive answers/segments, etc.) in a standard edition search engine. Please follow https://cloud.google.com/generative-ai-app-builder/docs/enterprise-edition#toggle-enterprise to enable Enterprise edition. Enterprise edition can be only enabled at the engine / app level. If the search request is against a data store, please update the serving config in the search request to use the engine/app ID instead, in the format of: "projects/*/locations/*/collections/*/engines/*/servingConfigs/*".

In [ ]:
retriever = VertexAISearchRetriever(
    project_id=PROJECT_ID,
    location_id=LOCATION,
    data_store_id=DATA_STORE_ID,
    max_documents=3,
    max_extractive_answer_count=3,
    get_extractive_answers=True,
)

result = retriever.invoke(search_query)
for doc in result:
    print(doc)

### Dual search

In [ ]:
from langchain_google_community import (
    VertexAIMultiTurnSearchRetriever,
    VertexAISearchRetriever,
)

# LangChain Retriever Configuration
SEARCH_ENGINE_ID = VERTEX_AI_SEARCH_APP_ID  # Set to your search app ID
DATA_STORE_ID = DATASTORE_ID  # DATASTORE_NAME # DATASTORE_ID
retriever = VertexAISearchRetriever(
    project_id=PROJECT_ID,
    location_id=LOCATION,
    data_store_id=DATA_STORE_ID,
    max_documents=3,
)

def search_with_langchain(query):
    """Search using LangChain VertexAISearchRetriever."""
    try:
        result = retriever.invoke(query)
        
        if result:
            # Combine content from all retrieved documents
            combined_content = ""
            sources = []
            
            for i, doc in enumerate(result):
                combined_content += f"\n--- Document {i+1} ---\n"
                combined_content += doc.page_content
                combined_content += "\n"
                
                # Extract source info
                if hasattr(doc, 'metadata') and 'title' in doc.metadata:
                    sources.append(doc.metadata['title'])
                else:
                    sources.append(f"Document {i+1}")
            
            return {
                'response': combined_content,
                'sources': sources,
                'num_sources': len(result)
            }
        else:
            return {
                'response': "No documents found",
                'sources': [],
                'num_sources': 0
            }
    except Exception as e:
        return {
            'response': f"Error: {str(e)}",
            'sources': [],
            'num_sources': 0
        }

def test_golden_set_dual_search(golden_set_df, num_tests=None, query_types=None):
    """
    Test both Discovery Engine and LangChain search methods with golden dataset.
    """
    
    if golden_set_df is None:
        print("❌ No golden set available for testing")
        return
    
    print("🧪 DUAL SEARCH EVALUATION (Discovery Engine vs LangChain)")
    print("=" * 80)
    
    # Filter by query types if specified
    test_df = golden_set_df.copy()
    if query_types:
        test_df = test_df[test_df['query_type'].isin(query_types)]
        print(f"📋 Filtering for query types: {query_types}")
    
    # Limit number of tests if specified
    if num_tests:
        test_df = test_df.head(num_tests)
        print(f"📊 Testing first {num_tests} queries")
    
    print(f"\n🎯 Running {len(test_df)} evaluation queries with both search methods...\n")
    
    results = []
    
    for idx, row in test_df.iterrows():
        query_id = row.get('id', f"Q-{idx:03d}")
        query_en = row.get('query_en', '')
        query_id_lang = row.get('query_id', '')
        query_type = row.get('query_type', 'Unknown')
        expected_answer = row.get('expected_answer', '')
        difficulty = row.get('difficulty', 'Unknown')
        
        print(f"\n🔍 {query_id} - {query_type} ({difficulty})")
        print(f"📝 Query (EN): {query_en}")
        if query_id_lang:
            print(f"📝 Query (ID): {query_id_lang}")
        print(f"🎯 Expected: {expected_answer}")
        print("-" * 60)
        
        try:
            # METHOD 1: Discovery Engine Search
            print("🔍 Method 1: Discovery Engine Search...")
            start_time = time.time()
            
            discovery_search_response = search_sample(
                PROJECT_ID, LOCATION, VERTEX_AI_SEARCH_APP_ID, query_en
            )
            
            discovery_time = time.time() - start_time
            
            # Extract Discovery Engine response
            if hasattr(discovery_search_response, 'summary') and discovery_search_response.summary:
                summary_response = discovery_search_response.summary.summary_with_metadata
                discovery_response = summary_response.summary if summary_response else "No summary available"
                
                discovery_references = []
                if summary_response and hasattr(summary_response, 'references'):
                    discovery_references = [ref.title for ref in summary_response.references]
                
                discovery_num_sources = len(discovery_references)
            else:
                discovery_response = "No response from Discovery Engine"
                discovery_references = []
                discovery_num_sources = 0
            
            print(f"🤖 Discovery Engine Response ({discovery_time:.2f}s):")
            print(discovery_response[:300] + "..." if len(discovery_response) > 300 else discovery_response)
            
            # METHOD 2: LangChain Search
            print(f"\n🔍 Method 2: LangChain Retrieval...")
            start_time = time.time()
            
            langchain_result = search_with_langchain(query_en)
            langchain_time = time.time() - start_time
            
            langchain_response = langchain_result['response']
            langchain_sources = langchain_result['sources'] 
            langchain_num_sources = langchain_result['num_sources']
            
            print(f"🤖 LangChain Response ({langchain_time:.2f}s):")
            print(langchain_response[:300] + "..." if len(langchain_response) > 300 else langchain_response)
            
            # ANALYZE BOTH RESPONSES
            # Discovery Engine Analysis
            discovery_quality = analyze_discovery_response_quality(discovery_response, expected_answer)
            discovery_score = calculate_discovery_quality_score(discovery_quality, discovery_num_sources)
            
            if not discovery_quality['apologetic'] and discovery_quality['specific_content'] and discovery_num_sources > 0:
                discovery_status = "✅ EXCELLENT"
            elif not discovery_quality['apologetic'] and discovery_quality['found_information']:
                discovery_status = "✅ GOOD"
            elif discovery_quality['helpful'] and not discovery_quality['apologetic']:
                discovery_status = "⚠️ PARTIAL"
            else:
                discovery_status = "❌ POOR"
            
            # LangChain Analysis  
            langchain_quality = analyze_langchain_response_quality(langchain_response, expected_answer)
            langchain_score = calculate_langchain_quality_score(langchain_quality, langchain_num_sources)
            
            if langchain_quality['contains_relevant_info'] and langchain_quality['specific_content'] and langchain_num_sources > 0:
                langchain_status = "✅ EXCELLENT"
            elif langchain_quality['contains_relevant_info'] and langchain_num_sources > 0:
                langchain_status = "✅ GOOD"
            elif langchain_quality['contains_relevant_info']:
                langchain_status = "⚠️ PARTIAL"
            else:
                langchain_status = "❌ POOR"
            
            print(f"\n📊 COMPARISON:")
            print(f"  Discovery Engine: {discovery_status} (Score: {discovery_score:.1f}, Sources: {discovery_num_sources})")
            print(f"  LangChain:        {langchain_status} (Score: {langchain_score:.1f}, Sources: {langchain_num_sources})")
            
            # Determine winner
            if discovery_score > langchain_score:
                winner = "Discovery Engine"
                score_diff = discovery_score - langchain_score
            elif langchain_score > discovery_score:
                winner = "LangChain"
                score_diff = langchain_score - discovery_score
            else:
                winner = "Tie"
                score_diff = 0
                
            print(f"🏆 Winner: {winner}" + (f" (+{score_diff:.1f})" if score_diff > 0 else ""))
            
            # Store results
            results.append({
                'query_id': query_id,
                'query_en': query_en,
                'query_type': query_type,
                'difficulty': difficulty,
                'expected_answer': expected_answer,
                
                # Discovery Engine Results
                'discovery_response': discovery_response,
                'discovery_time': discovery_time,
                'discovery_sources': discovery_references,
                'discovery_num_sources': discovery_num_sources,
                'discovery_status': discovery_status,
                'discovery_score': discovery_score,
                
                # LangChain Results
                'langchain_response': langchain_response,
                'langchain_time': langchain_time,
                'langchain_sources': langchain_sources,
                'langchain_num_sources': langchain_num_sources,
                'langchain_status': langchain_status,
                'langchain_score': langchain_score,
                
                # Comparison
                'winner': winner,
                'score_difference': abs(score_diff)
            })
            
        except Exception as e:
            print(f"❌ Error processing query: {e}")
            results.append({
                'query_id': query_id,
                'query_en': query_en,
                'query_type': query_type,
                'difficulty': difficulty,
                'expected_answer': expected_answer,
                'discovery_response': f"ERROR: {e}",
                'discovery_time': 0,
                'discovery_sources': [],
                'discovery_num_sources': 0,
                'discovery_status': "❌ ERROR",
                'discovery_score': 0,
                'langchain_response': f"ERROR: {e}",
                'langchain_time': 0,
                'langchain_sources': [],
                'langchain_num_sources': 0,
                'langchain_status': "❌ ERROR",
                'langchain_score': 0,
                'winner': "None",
                'score_difference': 0
            })
        
        print("\n" + "=" * 80)
        time.sleep(1)  # Brief pause between queries
    
    # COMPREHENSIVE RESULTS ANALYSIS
    results_df = pd.DataFrame(results)
    
    print(f"\n📊 DUAL SEARCH EVALUATION SUMMARY")
    print("=" * 60)
    print(f"Total Queries Tested: {len(results_df)}")
    
    # Overall Winner Analysis
    discovery_wins = len(results_df[results_df['winner'] == 'Discovery Engine'])
    langchain_wins = len(results_df[results_df['winner'] == 'LangChain'])  
    ties = len(results_df[results_df['winner'] == 'Tie'])
    
    print(f"\n🏆 OVERALL WINNER ANALYSIS:")
    print(f"  Discovery Engine wins: {discovery_wins}/{len(results_df)} ({(discovery_wins/len(results_df)*100):.1f}%)")
    print(f"  LangChain wins: {langchain_wins}/{len(results_df)} ({(langchain_wins/len(results_df)*100):.1f}%)")
    print(f"  Ties: {ties}/{len(results_df)} ({(ties/len(results_df)*100):.1f}%)")
    
    # Performance Metrics Comparison
    print(f"\n📈 PERFORMANCE METRICS COMPARISON:")
    discovery_avg_score = results_df['discovery_score'].mean()
    langchain_avg_score = results_df['langchain_score'].mean()
    discovery_avg_time = results_df['discovery_time'].mean()
    langchain_avg_time = results_df['langchain_time'].mean()
    
    print(f"  Average Quality Score:")
    print(f"    Discovery Engine: {discovery_avg_score:.1f}/10")
    print(f"    LangChain:        {langchain_avg_score:.1f}/10")
    
    print(f"  Average Response Time:")
    print(f"    Discovery Engine: {discovery_avg_time:.2f}s")
    print(f"    LangChain:        {langchain_avg_time:.2f}s")
    
    # Query Type Performance
    print(f"\n📋 PERFORMANCE BY QUERY TYPE:")
    for qtype in results_df['query_type'].unique():
        type_results = results_df[results_df['query_type'] == qtype]
        
        discovery_type_avg = type_results['discovery_score'].mean()
        langchain_type_avg = type_results['langchain_score'].mean()
        
        discovery_type_wins = len(type_results[type_results['winner'] == 'Discovery Engine'])
        langchain_type_wins = len(type_results[type_results['winner'] == 'LangChain'])
        
        print(f"  {qtype}:")
        print(f"    Discovery Engine: Avg {discovery_type_avg:.1f}, Wins {discovery_type_wins}/{len(type_results)}")
        print(f"    LangChain:        Avg {langchain_type_avg:.1f}, Wins {langchain_type_wins}/{len(type_results)}")
    
    # Biggest Score Differences
    print(f"\n📊 LARGEST PERFORMANCE GAPS:")
    biggest_gaps = results_df.nlargest(3, 'score_difference')[['query_id', 'query_en', 'winner', 'score_difference']]
    for _, query in biggest_gaps.iterrows():
        print(f"  {query['query_id']}: {query['query_en'][:40]}... → {query['winner']} (+{query['score_difference']:.1f})")
    
    return results_df

def analyze_langchain_response_quality(response_text, expected_answer):
    """Analyze quality of LangChain retrieval response."""
    
    response_lower = response_text.lower()
    
    # Check for error messages or no content
    error_phrases = ["error:", "no documents found", "no information"]
    has_error = any(phrase in response_lower for phrase in error_phrases)
    
    # Check if response contains relevant information
    contains_relevant_info = len(response_text.strip()) > 50 and not has_error
    
    # Check for specific content mentions
    specific_indicators = [
        "timestamp", "episode", "scene", "character", "dialogue", 
        "minute", "second", "around", "at", "says", "mentions"
    ]
    specific_content = any(indicator in response_lower for indicator in specific_indicators)
    
    # Check for video/episode references
    video_indicators = ["video", "episode", "series", "mp4", "content"]
    mentions_video = any(indicator in response_lower for indicator in video_indicators)
    
    return {
        'has_error': has_error,
        'contains_relevant_info': contains_relevant_info,
        'specific_content': specific_content,
        'mentions_video': mentions_video,
        'response_length': len(response_text)
    }

def calculate_langchain_quality_score(quality_analysis, num_sources):
    """Calculate quality score for LangChain response."""
    
    score = 0
    
    # Base scoring
    if not quality_analysis['has_error']:
        score += 2
        
    if quality_analysis['contains_relevant_info']:
        score += 4
        
    if quality_analysis['specific_content']:
        score += 2
        
    if quality_analysis['mentions_video']:
        score += 1
    
    # Bonus for retrieved sources
    if num_sources > 0:
        score += min(num_sources * 0.5, 2)
    
    return min(score, 10)

# Quick test functions
def quick_dual_test():
    """Quick dual search test with first 5 queries."""
    if golden_set is not None:
        return test_golden_set_dual_search(golden_set, num_tests=5)
    else:
        print("❌ Golden set not available")
        return None

def test_news_dual_search():
    """Test both search methods on news queries only."""
    if golden_set is not None:
        return test_golden_set_dual_search(golden_set, query_types=['News'])
    else:
        print("❌ Golden set not available")
        return None

print("🧪 DUAL SEARCH EVALUATION READY!")
print("\n💡 Available functions:")
print("test_golden_set_dual_search(golden_set, num_tests=5)  # Test first 5 with both methods")
print("test_news_dual_search()  # Test news queries only with both methods")  
print("quick_dual_test()  # Quick 5-query dual comparison")

In [ ]:
results = test_golden_set_dual_search(golden_set, num_tests=30)


### ONLY LANGCHAIN

In [43]:
# LangChain Search Evaluation
from langchain_google_community import (
    VertexAIMultiTurnSearchRetriever,
    VertexAISearchRetriever,
)

# LangChain Retriever Configuration
SEARCH_ENGINE_ID = VERTEX_AI_SEARCH_APP_ID  # Set to your search app ID
DATA_STORE_ID = DATASTORE_ID  # DATASTORE_NAME # DATASTORE_ID
retriever = VertexAISearchRetriever(
    project_id=PROJECT_ID,
    location_id=LOCATION,
    data_store_id=DATA_STORE_ID,
    max_documents=5,  # Increased for better coverage
)

def search_with_langchain_detailed(query):
    """Search using LangChain with detailed response processing."""
    try:
        result = retriever.invoke(query)
        
        if result:
            # Combine content from all retrieved documents
            combined_content = ""
            sources = []
            
            for i, doc in enumerate(result):
                combined_content += doc.page_content + "\n"
                
                # Extract source info
                if hasattr(doc, 'metadata') and 'title' in doc.metadata:
                    sources.append(doc.metadata['title'])
                else:
                    sources.append(f"Document {i+1}")
            
            return {
                'response': combined_content.strip(),
                'sources': sources,
                'num_sources': len(result),
                'raw_docs': result
            }
        else:
            return {
                'response': "No documents found",
                'sources': [],
                'num_sources': 0,
                'raw_docs': []
            }
    except Exception as e:
        return {
            'response': f"Error: {str(e)}",
            'sources': [],
            'num_sources': 0,
            'raw_docs': []
        }

def test_golden_set_with_langchain(golden_set_df, num_tests=None, query_types=None):
    """
    Test LangChain search system with golden dataset using same evaluation metrics.
    """
    
    if golden_set_df is None:
        print("❌ No golden set available for testing")
        return
    
    print("🧪 GOLDEN SET EVALUATION WITH LANGCHAIN")
    print("=" * 80)
    
    # Filter by query types if specified
    test_df = golden_set_df.copy()
    if query_types:
        test_df = test_df[test_df['query_type'].isin(query_types)]
        print(f"📋 Filtering for query types: {query_types}")
    
    # Limit number of tests if specified
    if num_tests:
        test_df = test_df.head(num_tests)
        print(f"📊 Testing first {num_tests} queries")
    
    print(f"\n🎯 Running {len(test_df)} evaluation queries...\n")
    
    results = []
    
    for idx, row in test_df.iterrows():
        query_id = row.get('id', f"Q-{idx:03d}")
        query_en = row.get('query_en', '')
        query_id_lang = row.get('query_id', '')
        query_type = row.get('query_type', 'Unknown')
        expected_answer = row.get('expected_answer', '')
        difficulty = row.get('difficulty', 'Unknown')
        
        print(f"\n🔍 {query_id} - {query_type} ({difficulty})")
        
        query_en = query_en + " ---  give get the time stamp of the scene " 
        print(f"📝 Query (EN): {query_en}")
        
       
        if query_id_lang:
            query_id_lang  = query_id_lang + " berikan stempel waktu dari adegan tersebut"
            print(f"📝 Query (ID): {query_id_lang}")
        print(f"🎯 Expected: {expected_answer}")
        print("-" * 60)
        
        try:
            start_time = time.time()
            
            # Use LangChain search
            print("🔍 Searching with LangChain...")
            
            langchain_result = search_with_langchain_detailed(query_en)
            response_time = time.time() - start_time
            
            langchain_response = langchain_result['response']
            sources = langchain_result['sources']
            num_sources = langchain_result['num_sources']
            
            print(f"🤖 LangChain Response ({response_time:.2f}s):")
            print(langchain_response[:1000] + "..." if len(langchain_response) > 1000 else langchain_response)
            
            # Analyze response quality using existing framework
            quality_analysis = analyze_langchain_response_quality(langchain_response, expected_answer)
            
            # Quality status determination
            if quality_analysis['contains_relevant_info'] and quality_analysis['specific_content'] and num_sources > 0:
                quality_status = "✅ EXCELLENT"
            elif quality_analysis['contains_relevant_info'] and num_sources > 0:
                quality_status = "✅ GOOD"
            elif quality_analysis['contains_relevant_info']:
                quality_status = "⚠️ PARTIAL"
            else:
                quality_status = "❌ POOR"
            
            print(f"\n📊 Response Quality: {quality_status}")
            
            # Show quality indicators
            if quality_analysis['has_error']:
                print("   - ❌ Response contains errors or no content")
            if quality_analysis['contains_relevant_info']:
                print("   - ✅ Contains relevant information")
            if quality_analysis['specific_content']:
                print("   - ✅ Contains specific content details")
            if quality_analysis['mentions_video']:
                print("   - ✅ Mentions video/episode information")
            if quality_analysis['has_timestamps']:
                print("   - ✅ Contains timestamp information")
            
            print(f"\n📄 Sources Retrieved: {num_sources} documents found")
            if num_sources > 0:
                for i, source in enumerate(sources[:3], 1):
                    print(f"  {i}. {source}")
            else:
                print("  ⚠️ No documents retrieved")
            
            # Calculate quality score
            quality_score = calculate_langchain_quality_score(quality_analysis, num_sources)
            
            # Store results
            results.append({
                'query_id': query_id,
                'query_en': query_en,
                'query_type': query_type,
                'difficulty': difficulty,
                'expected_answer': expected_answer,
                'langchain_response': langchain_response,
                'response_time': response_time,
                'num_sources': num_sources,
                'sources': sources,
                'quality_status': quality_status,
                'quality_score': quality_score,
                'contains_relevant_info': quality_analysis['contains_relevant_info'],
                'has_error': quality_analysis['has_error'],
                'specific_content': quality_analysis['specific_content'],
                'mentions_video': quality_analysis['mentions_video'],
                'has_timestamps': quality_analysis['has_timestamps']
            })
            
        except Exception as e:
            print(f"❌ Error processing query: {e}")
            results.append({
                'query_id': query_id,
                'query_en': query_en,
                'query_type': query_type,
                'difficulty': difficulty,
                'expected_answer': expected_answer,
                'langchain_response': f"ERROR: {e}",
                'response_time': 0,
                'num_sources': 0,
                'sources': [],
                'quality_status': "❌ ERROR",
                'quality_score': 0,
                'contains_relevant_info': False,
                'has_error': True,
                'specific_content': False,
                'mentions_video': False,
                'has_timestamps': False
            })
        
        print("\n" + "=" * 80)
        time.sleep(1)  # Brief pause between queries
    
    # Results summary using same format as Discovery Engine evaluation
    results_df = pd.DataFrame(results)
    
    print(f"\n📊 LANGCHAIN EVALUATION SUMMARY")
    print("=" * 60)
    print(f"Total Queries Tested: {len(results_df)}")
    
    # Quality breakdown
    excellent = len(results_df[results_df['quality_status'] == '✅ EXCELLENT'])
    good = len(results_df[results_df['quality_status'] == '✅ GOOD'])
    partial = len(results_df[results_df['quality_status'] == '⚠️ PARTIAL'])
    poor = len(results_df[results_df['quality_status'] == '❌ POOR'])
    errors = len(results_df[results_df['quality_status'] == '❌ ERROR'])
    
    print(f"\nQuality Distribution:")
    print(f"  Excellent: {excellent}/{len(results_df)} ({(excellent/len(results_df)*100):.1f}%)")
    print(f"  Good: {good}/{len(results_df)} ({(good/len(results_df)*100):.1f}%)")
    print(f"  Partial: {partial}/{len(results_df)} ({(partial/len(results_df)*100):.1f}%)")
    print(f"  Poor: {poor}/{len(results_df)} ({(poor/len(results_df)*100):.1f}%)")
    if errors > 0:
        print(f"  Errors: {errors}/{len(results_df)} ({(errors/len(results_df)*100):.1f}%)")
    
    print(f"\nPerformance Metrics:")
    print(f"  Average quality score: {results_df['quality_score'].mean():.1f}/10")
    print(f"  Average response time: {results_df['response_time'].mean():.2f}s")
    
    # Source retrieval stats
    with_sources = len(results_df[results_df['num_sources'] > 0])
    print(f"  Queries with sources: {with_sources}/{len(results_df)} ({(with_sources/len(results_df)*100):.1f}%)")
    
    # Query type breakdown
    print(f"\nPerformance by Query Type:")
    for qtype in results_df['query_type'].unique():
        type_results = results_df[results_df['query_type'] == qtype]
        avg_score = type_results['quality_score'].mean()
        success_rate = len(type_results[type_results['contains_relevant_info']]) / len(type_results) * 100
        print(f"  {qtype}: Avg Score {avg_score:.1f}, Success Rate {success_rate:.1f}%")
    
    # Show problematic queries (same as Discovery Engine evaluation)
    print(f"\n⚠️ Problematic Queries (Poor/Error):")
    poor_queries = results_df[results_df['quality_status'].isin(['❌ POOR', '❌ ERROR'])]
    for _, query in poor_queries.iterrows():
        print(f"  {query['query_id']}: {query['query_en'][:50]}... ({query['quality_status']})")
    
    # Show best performing queries
    print(f"\n🏆 Best Performing Queries:")
    top_queries = results_df.nlargest(3, 'quality_score')[['query_id', 'query_en', 'quality_score', 'quality_status']]
    for _, query in top_queries.iterrows():
        print(f"  {query['query_id']}: {query['query_en'][:50]}... (Score: {query['quality_score']:.1f})")
    
    return results_df

def analyze_langchain_response_quality(response_text, expected_answer):
    """Analyze quality of LangChain retrieval response."""
    
    response_lower = response_text.lower()
    
    # Check for error messages or no content
    error_phrases = ["error:", "no documents found", "no information available"]
    has_error = any(phrase in response_lower for phrase in error_phrases)
    
    # Check if response contains relevant information
    contains_relevant_info = len(response_text.strip()) > 50 and not has_error
    
    # Check for specific content mentions (enhanced for video content)
    specific_indicators = [
        "timestamp", "episode", "scene", "character", "dialogue", 
        "minute", "second", "around", "at", "says", "mentions",
        "galaksi", "aluna", "galaxy", "trump", "marcos"
    ]
    specific_content = any(indicator in response_lower for indicator in specific_indicators)
    
    # Check for video/episode references
    video_indicators = ["video", "episode", "series", "mp4", "content", "film"]
    mentions_video = any(indicator in response_lower for indicator in video_indicators)
    
    # Check for timestamp information
    timestamp_indicators = ["timestamp", "around", "at", "minute", "second", ":"]
    has_timestamps = any(indicator in response_lower for indicator in timestamp_indicators)
    
    return {
        'has_error': has_error,
        'contains_relevant_info': contains_relevant_info,
        'specific_content': specific_content,
        'mentions_video': mentions_video,
        'has_timestamps': has_timestamps,
        'response_length': len(response_text)
    }

def calculate_langchain_quality_score(quality_analysis, num_sources):
    """Calculate quality score for LangChain response (0-10)."""
    
    score = 0
    
    # Base scoring
    if not quality_analysis['has_error']:
        score += 2
        
    if quality_analysis['contains_relevant_info']:
        score += 3
        
    if quality_analysis['specific_content']:
        score += 2
        
    if quality_analysis['mentions_video']:
        score += 1
        
    if quality_analysis['has_timestamps']:
        score += 1
    
    # Bonus for retrieved sources
    if num_sources > 0:
        score += min(num_sources * 0.2, 1)  # Up to 1 bonus point for sources
    
    return min(score, 10)

# Quick test functions
def quick_langchain_test():
    """Quick LangChain test with first 5 queries."""
    if golden_set is not None:
        return test_golden_set_with_langchain(golden_set, num_tests=5)
    else:
        print("❌ Golden set not available")
        return None

def test_langchain_news_only():
    """Test LangChain on news queries only."""
    if golden_set is not None:
        return test_golden_set_with_langchain(golden_set, query_types=['News'])
    else:
        print("❌ Golden set not available")
        return None

print("🧪 LANGCHAIN GOLDEN SET EVALUATION READY!")
print("\n💡 Available functions:")
print("test_golden_set_with_langchain(golden_set, num_tests=5)  # Test first 5")
print("test_golden_set_with_langchain(golden_set)  # Test all queries")
print("test_langchain_news_only()  # Test news queries only")
print("quick_langchain_test()  # Quick 5-query test")

🧪 LANGCHAIN GOLDEN SET EVALUATION READY!

💡 Available functions:
test_golden_set_with_langchain(golden_set, num_tests=5)  # Test first 5
test_golden_set_with_langchain(golden_set)  # Test all queries
test_langchain_news_only()  # Test news queries only
quick_langchain_test()  # Quick 5-query test


/opt/conda/lib/python3.10/site-packages/langchain_google_community/vertex_ai_search.py:365: UserWarning: Beta features are configured but beta=False. The following beta features will be ignored:['custom_embedding_ratio']
  warnings.warn(


In [44]:
results = test_golden_set_with_langchain(golden_set)


🧪 GOLDEN SET EVALUATION WITH LANGCHAIN

🎯 Running 30 evaluation queries...


🔍 Q-001 - Soap opera (Easy)
📝 Query (EN): Aluna fall from horse ---  give get the time stamp of the scene 
📝 Query (ID): Aluna jatuh dari kuda berikan stempel waktu dari adegan tersebut
🎯 Expected: Episode 1&2 - around 00:54
------------------------------------------------------------
🔍 Searching with LangChain...
🤖 LangChain Response (1.81s):
*   **00:00 - 00:15:** The scene opens with a young woman, Aluna, in a white shirt, standing in a grassy field, shouting instructions with her arms raised. She has a panicked expression. She is trying to help a young man, Galaksi, who is on a black horse that is running uncontrollably down a dirt path towards her. The man, wearing a plaid shirt, is struggling to control the horse. Aluna gestures for him to jump, and as the horse gets close, she extends her hands. He grabs her hand, but the horse's momentum pulls her off her feet and onto the back of the horse, behind him